In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import json
import os
import glob

In [2]:
csv_file = "data-files/Food_Inspections.csv"
food_inspections_df = pd.read_csv(csv_file)
food_inspections_df.head()

fidf = food_inspections_df.rename(columns={'AKA Name':'name', 'Address':'address1'})
fidf = fidf.replace('&', 'AND', regex=True)
fidf = fidf.replace('-', ' ', regex=True)
fidf['name'] = fidf['name'].str.upper()
fidf['address1'] = fidf['address1'].str.upper()
fidf.head()

,Inspection ID,DBA Name,name,License #,Facility Type,Risk,address1,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2362453,"DELICIOUS PHO,INC.",DELICIOUS PHO,2695007.0,Restaurant,Risk 1 (High),5033 N ELSTON AVE,CHICAGO,IL,60630.0,02/18/2020,License,Not Ready,NaN,41.972349,-87.746825,"( 87.74682508578468, 41.97234928256036)"
1,2362427,SUBWAY SANDWICH AND SALAD,SUBWAY SANDWICH AND SALAD,1299198.0,NaN,Risk 3 (Low),4640 A W DIVERSEY AVE,CHICAGO,IL,60639.0,02/18/2020,Canvass,Out of Business,NaN,41.931682,-87.743565,"( 87.7435650331494, 41.93168226651076)"
2,2362426,REEF KITCHENS,REEF KITCHENS,2694910.0,NaN,Risk 1 (High),135 N KEDZIE,CHICAGO,IL,60612.0,02/18/2020,License,Not Ready,NaN,41.883905,-87.706059,"( 87.70605925083947, 41.88390509704504)"
3,2362400,GALLERY CAFE,NaN,2713470.0,NaN,Risk 1 (High),3734 N MILWAUKEE AVE,CHICAGO,IL,60641.0,02/14/2020,License,No Entry,NaN,41.948591,-87.739661,"( 87.73966083900571, 41.94859067460761)"
4,2362385,79TH BBQ,79TH BBQ,2616936.0,Restaurant,Risk 1 (High),11101 S HALSTED ST,CHICAGO,IL,60628.0,02/14/2020,Canvass,Pass,57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAIN...,41.692267,-87.642230,"( 87.64222983410407, 41.692267088936674)"


In [3]:
pd.set_option('display.max_columns', None)

temp = pd.DataFrame()

path_to_json = 'data-files/json/' 

json_pattern = os.path.join(path_to_json, '*.json')
file_list = glob.glob(json_pattern)
for file in file_list:
    data = json.load(open(file))
    pd.read_json(file, lines=True)
    temp = temp.append(data["businesses"], ignore_index=True)

temp.head(10)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8oKRLk54AhYwEaBe7onYog,green-door-tavern-chicago,Green Door Tavern,https://s3-media4.fl.yelpcdn.com/bphoto/C6S3gr...,False,https://www.yelp.com/biz/green-door-tavern-chi...,451,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.0,"{'latitude': 41.8946215100706, 'longitude': -8...","[pickup, delivery]",$$,"{'address1': '678 N Orleans St', 'address2': '...",+13126645496,(312) 664-5496,3542.992311
1,bvB-S8FsKodF7qkfIfYI8g,pops-for-champagne-chicago,Pops for Champagne,https://s3-media2.fl.yelpcdn.com/bphoto/oa3hIh...,False,https://www.yelp.com/biz/pops-for-champagne-ch...,779,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,"{'latitude': 41.8926209396105, 'longitude': -8...",[delivery],$$$,"{'address1': '601 N State St', 'address2': '',...",+13122667677,(312) 266-7677,4355.300127
2,Sx0CL_h4JitfC1qlsKAuMw,daebak-korean-bbq-chicago-3,Daebak Korean BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/dbGfWE...,False,https://www.yelp.com/biz/daebak-korean-bbq-chi...,492,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"{'latitude': 41.85477, 'longitude': -87.63257}",[],$$,"{'address1': '2017 S Wells St', 'address2': 'F...",+13126313913,(312) 631-3913,6730.639121
3,UQJ6tKY4Uz6P-7d7oych2w,rock-bottom-restaurant-and-brewery-chicago,Rock Bottom Restaurant & Brewery,https://s3-media1.fl.yelpcdn.com/bphoto/NdEOZo...,False,https://www.yelp.com/biz/rock-bottom-restauran...,1047,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,"{'latitude': 41.8915042, 'longitude': -87.6283...","[pickup, delivery]",$$,"{'address1': '1 W Grand Ave', 'address2': '', ...",+13127559339,(312) 755-9339,4361.493476
4,-C_O2sunRZD_sgWtCIpuAQ,cellars-bar-and-grill-chicago,Cellars Bar and Grill,https://s3-media2.fl.yelpcdn.com/bphoto/iMK0Pk...,False,https://www.yelp.com/biz/cellars-bar-and-grill...,579,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 41.98922, 'longitude': -87.66065}","[pickup, delivery]",$$,"{'address1': '5900 N Broadway', 'address2': No...",+17739441208,(773) 944-1208,9453.587931
5,Hs3b3jbwPQ88qtycbcR65w,lucias-gourmet-italian-cuisine-chicago,Lucia's Gourmet Italian Cuisine,https://s3-media3.fl.yelpcdn.com/bphoto/ryouGA...,False,https://www.yelp.com/biz/lucias-gourmet-italia...,337,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 41.91026, 'longitude': -87.67359}","[pickup, delivery]",$$,"{'address1': '1825 W North Ave', 'address2': '...",+17732929700,(773) 292-9700,670.843088
6,W8-W5b8kZYCM7AnN7Ct7TA,little-beet-table-chicago,Little Beet Table,https://s3-media1.fl.yelpcdn.com/bphoto/HeAg5r...,False,https://www.yelp.com/biz/little-beet-table-chi...,340,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 41.8980946545345, 'longitude': -8...","[pickup, delivery]",$$,"{'address1': '845 N State St', 'address2': 'St...",+13125498600,(312) 549-8600,4188.613399
7,ntLthO5cQDnwn-vKiT3WxA,zaytune-mediterranean-grill-chicago-4,Zaytune Mediterranean Grill,https://s3-media3.fl.yelpcdn.com/bphoto/LhMDJI...,False,https://www.yelp.com/biz/zaytune-mediterranean...,427,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.0,"{'latitude': 41.8370666503906, 'longitude': -8...","[pickup, delivery]",$$,"{'address1': '3129 S Morgan St', 'address2': '...",+17732546300,(773) 254-6300,7892.567636
8,7xpf5VhK73NN3gcb4gbMCA,epic-burger-chicago,Epic Burger,https://s3-media2.fl.yelpcdn.com/bphoto/jWmqBu...,False,https://www.yelp.com/biz/epic-burger-chicago?a...,794,"[{'alias': 'burgers', 'title': 'Burgers'}]",3.5,"{'latitude': 41.8752826, 'longitude': -87.6274...","[pickup, delivery]",$$,"{'address1': '517 S State St', 'address2': '',...",+13129131373,(312) 913-1373,5352.217033
9,S96imo1X3KP-CPBd7rhjWw,smack-dab-chicago-12,Smack Dab,https://s3-media3.fl.yelpcdn.com/bphoto/3UCwPz...,False,https://www.yelp.com/biz/smack-dab-chicago-12?...,221,"[{'alias': 'breakfast_brunch'

In [4]:
df2 = temp[['name', 'review_count', 'rating', 'price', 'coordinates', 'location']].copy()
df2

,name,review_count,rating,price,coordinates,location
0,Green Door Tavern,451,4.0,$$,"{'latitude': 41.8946215100706, 'longitude': -8...","{'address1': '678 N Orleans St', 'address2': '..."
1,Pops for Champagne,779,3.5,$$$,"{'latitude': 41.8926209396105, 'longitude': -8...","{'address1': '601 N State St', 'address2': '',..."
2,Daebak Korean BBQ,492,4.0,$$,"{'latitude': 41.85477, 'longitude': -87.63257}","{'address1': '2017 S Wells St', 'address2': 'F..."
3,Rock Bottom Restaurant & Brewery,1047,3.5,$$,"{'latitude': 41.8915042, 'longitude': -87.6283...","{'address1': '1 W Grand Ave', 'address2': '', ..."
4,Cellars Bar and Grill,579,4.0,$$,"{'latitude': 41.98922, 'longitude': -87.66065}","{'address1': '5900 N Broadway', 'address2': No..."
...,...,...,...,...,...,...
995,Over Easy Cafe,1464,4.5,$$,"{'latitude': 41.9717208, 'longitude': -87.6790...","{'address1': '4943 N Damen Ave', 'address2': N..."
996,Volare Ristorante Italiano,2083,4.0,$$,"{'latitude': 41.8915901798304, 'longitude': -8...","{'address1': '201 E Grand Ave', 'address2': ''..."
997,Do-Rite Donuts & Chicken,1141,4.5,$,"{'latitude': 41.894028, 'longitude': -87.6213298}","{'address1': '233 E Erie St', 'address2': '', ..."
998,Siena Tavern,1754,4.0,$$$,"{'latitude': 41.8891395940387, 'longitude': -8...","{'address1': '51 W Kinzie St', 'address2': '',..."


In [5]:
latitude=[]
longitude=[]
address1=[]
address2=[]
address3=[]
city=[]
zip_code=[]
country=[]
state=[]

for k,v in df2.iterrows():
    latitude.append(v.coordinates['latitude'])
    longitude.append(v.coordinates['longitude'])
    address1.append(v.location['address1'])
    address2.append(v.location['address2'])
    address3.append(v.location['address3'])
    city.append(v.location['city'])
    zip_code.append(v.location['zip_code'])
    country.append(v.location['country'])
    state.append(v.location['state'])

In [6]:
df2['latitude'] = latitude
df2['longitude'] = longitude
df2['address1'] = address1
df2['address2'] = address2
df2['address3'] = address3
df2['city'] = city
df2['zip_code'] = zip_code
df2['country'] = country
df2['state'] = state

In [7]:
df2.head()

,name,review_count,rating,price,coordinates,location,latitude,longitude,address1,address2,address3,city,zip_code,country,state
0,Green Door Tavern,451,4.0,$$,"{'latitude': 41.8946215100706, 'longitude': -8...","{'address1': '678 N Orleans St', 'address2': '...",41.894622,-87.637365,678 N Orleans St,,,Chicago,60654,US,IL
1,Pops for Champagne,779,3.5,$$$,"{'latitude': 41.8926209396105, 'longitude': -8...","{'address1': '601 N State St', 'address2': '',...",41.892621,-87.627911,601 N State St,,,Chicago,60654,US,IL
2,Daebak Korean BBQ,492,4.0,$$,"{'latitude': 41.85477, 'longitude': -87.63257}","{'address1': '2017 S Wells St', 'address2': 'F...",41.854770,-87.632570,2017 S Wells St,Fl 2,None,Chicago,60616,US,IL
3,Rock Bottom Restaurant & Brewery,1047,3.5,$$,"{'latitude': 41.8915042, 'longitude': -87.6283...","{'address1': '1 W Grand Ave', 'address2': '', ...",41.891504,-87.628364,1 W Grand Ave,,,Chicago,60610,US,IL
4,Cellars Bar and Grill,579,4.0,$$,"{'latitude': 41.98922, 'longitude': -87.66065}","{'address1': '5900 N Broadway', 'address2': No...",41.989220,-87.660650,5900 N Broadway,None,,Chicago,60660,US,IL


In [9]:
df3 = df2[['name', 'review_count', 'rating', 'price', 'latitude', 'longitude', 'address1', 'address2', 'address3', 'city',
       'zip_code', 'country', 'state']].copy()
df3 = df3.replace('&', 'AND', regex=True)
df3 = df3.replace('-', ' ', regex=True)
df3['name'] = df3['name'].str.upper()
df3['address1'] = df3['address1'].str.upper()
df3.head()

,name,review_count,rating,price,latitude,longitude,address1,address2,address3,city,zip_code,country,state
0,GREEN DOOR TAVERN,451,4.0,$$,41.894622,-87.637365,678 N ORLEANS ST,,,Chicago,60654,US,IL
1,POPS FOR CHAMPAGNE,779,3.5,$$$,41.892621,-87.627911,601 N STATE ST,,,Chicago,60654,US,IL
2,DAEBAK KOREAN BBQ,492,4.0,$$,41.854770,-87.632570,2017 S WELLS ST,Fl 2,None,Chicago,60616,US,IL
3,ROCK BOTTOM RESTAURANT AND BREWERY,1047,3.5,$$,41.891504,-87.628364,1 W GRAND AVE,,,Chicago,60610,US,IL
4,CELLARS BAR AND GRILL,579,4.0,$$,41.989220,-87.660650,5900 N BROADWAY,None,,Chicago,60660,US,IL


In [10]:
df3['address1']= df3['address1'].str.strip()
fidf['address1']= fidf['address1'].str.strip()

In [11]:
merged_df = pd.merge(fidf, df3, on=['name', 'address1'], how='inner')

In [19]:
merged_df.columns
restaurant_violations_df = merged_df[['name', 'Facility Type',
       'Risk', 'address1', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results', 'Violations', 'Latitude', 'Longitude',
       'review_count', 'rating', 'price']].copy()
restaurant_violations_df = restaurant_violations_df.rename(columns={"Facility Type": "facility_type", "Risk": "risk", "address1": "address", "City": "city", "State": "state", "Zip": "zip", "Inspection Date": "inspection_date", "Inspection Type": "inspection_type", "Results": "results", "Violations": "violations", "Latitude": "latitude", "Longitude": "longitude"})
restaurant_violations_df.head()

,name,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,review_count,rating,price
0,BOPNGRILL,Restaurant,Risk 1 (High),6604 N SHERIDAN RD,CHICAGO,IL,60626.0,02/13/2020,Canvass Re Inspection,Pass,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,42.002726,-87.660938,824,4.0,$$
1,BOPNGRILL,Restaurant,Risk 1 (High),6604 N SHERIDAN RD,CHICAGO,IL,60626.0,02/05/2020,Canvass,Fail,25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCO...,42.002726,-87.660938,824,4.0,$$
2,BOPNGRILL,Restaurant,Risk 1 (High),6604 N SHERIDAN RD,CHICAGO,IL,60626.0,07/29/2011,License Re Inspection,Pass,NaN,42.002726,-87.660938,824,4.0,$$
3,BOPNGRILL,Restaurant,Risk 1 (High),6604 N SHERIDAN RD,CHICAGO,IL,60626.0,03/26/2019,Canvass Re Inspection,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,42.002726,-87.660938,824,4.0,$$
4,BOPNGRILL,Restaurant,Risk 1 (High),6604 N SHERIDAN RD,CHICAGO,IL,60626.0,03/19/2019,Canvass,Fail,16. FOOD CONTACT SURFACES: CLEANED AND SANITIZ...,42.002726,-87.660938,824,4.0,$$


In [22]:
USERNAME = "postgres"
PASSWORD = "password"
DATABASE = "restaurant_violations_db"
rds_connection_string = f"{USERNAME}:{PASSWORD}@localhost:5432/{DATABASE}"
engine = create_engine(f'postgresql://{rds_connection_string}')
restaurant_violations_df.to_sql(name='restaurant_violations', con=engine, if_exists='append', index=True)